In [1]:
!pip install pandarallel
import pandas as pd
pd.set_option("max_colwidth", 600)
from bs4 import BeautifulSoup
import requests

from pandarallel import pandarallel


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandarallel: filename=pandarallel-1.6.1-py3-none-any.whl size=16265 sha256=d98d1975dbc6730877e68ec82e1277dd7ab3041e1b37c0be9fc45bbf82885064
  Stored in directory: /root/.cache/pip/wheels/e8/81/8f/29aafd7d671a4e9db4e976d71728870173f0b3f48ccba0bc32
Successfully built pandarallel


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
def extract_urls_topic(url):
    try:
      coverpage = requests.get(url).content
      soup = BeautifulSoup(coverpage, 'html.parser')
     
      rows = []
     
      for link in soup.find_all('h2', itemprop='name'):   
        rows.append('https://abi.bo'+link.a['href'])

      for link in soup.find_all('ol', class_='clearfix'):   
        rows.append('https://abi.bo'+link.a['href'])

    except:
      rows = 'error'
      
    return rows

In [23]:
extract_urls_topic('https://abi.bo/index.php/noticias/politica')

['https://abi.bo/index.php/noticias/politica/24526-gremiales-de-el-alto-rechazan-paro-en-santa-cruz',
 'https://abi.bo/index.php/noticias/politica/24523-alcalde-de-bolpebra-pide-a-camacho-no-politizar-el-censo-con-amenazas',
 'https://abi.bo/index.php/noticias/politica/24521-legisladora-afirma-que-camacho-busca-desestabilizar-al-gobierno-nacional-con-anuncio-de-paro',
 'https://abi.bo/index.php/noticias/politica/24517-bloque-oriente-y-organizaciones-sociales-de-santa-cruz-rechazan-el-paro-civico-del-25-de-julio',
 'https://abi.bo/index.php/noticias/politica/24512-paro-civico-en-santa-cruz-generara-perdidas-por-us-32-millones',
 'https://abi.bo/index.php/noticias/politica/24481-presidente-de-diputados-ratifica-que-designacion-del-defensor-es-competencia-de-la-asamblea-legislativa-3',
 'https://abi.bo/index.php/noticias/politica/24478-diputados-disponen-de-un-millon-de-bolivianos-para-socializar-leyes-en-260-foros-ciudadanos',
 'https://abi.bo/index.php/noticias/politica/24476-concluyo-e

In [24]:
pandarallel.initialize(nb_workers= 20, progress_bar=True)
df = pd.DataFrame(range(1,678),columns = ['page'])
url_str=''

#politica
for i in range(0,484) :
    url_str='https://abi.bo/index.php/noticias/politica?start='+str(i*7)
    df.page[i] = url_str

#inter
for i in range(0,194) :
    url_str='https://abi.bo/index.php/noticias/internacional?start='+str(i*7)
    df.page[i+484] = url_str

df['rows']=df['page'].parallel_apply(extract_urls_topic)
df = df[df.rows.str.len()!=0].reset_index(drop=True)
df = df.explode('rows', ignore_index=True)
df=df.drop_duplicates(subset='rows')
df = df.dropna()
print(df)
df.to_csv('/content/drive/MyDrive/Colab Notebooks/agencia/url.csv',index=False)

INFO: Pandarallel will run on 20 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


                                                            page  \
0             https://abi.bo/index.php/noticias/politica?start=0   
1             https://abi.bo/index.php/noticias/politica?start=0   
2             https://abi.bo/index.php/noticias/politica?start=0   
3             https://abi.bo/index.php/noticias/politica?start=0   
4             https://abi.bo/index.php/noticias/politica?start=0   
...                                                          ...   
5403  https://abi.bo/index.php/noticias/internacional?start=1337   
5405  https://abi.bo/index.php/noticias/internacional?start=1344   
5406  https://abi.bo/index.php/noticias/internacional?start=1344   
5407  https://abi.bo/index.php/noticias/internacional?start=1344   
5408  https://abi.bo/index.php/noticias/internacional?start=1344   

                                                                                                                                                           rows  
0                    